In [1]:
!pip  install syllables


In [24]:
! pip install openpyxl

                                              0.0/250.0 kB ? eta -:--:--
     ------------------------               163.8/250.0 kB 5.0 MB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 5.1 MB/s eta 0:00:00


In [2]:
!pip install nltk

# we installed syllables


# now , we have to import libraries

In [3]:
import os
import requests
import pandas as ps
import nltk #()
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import cmudict
from bs4 import BeautifulSoup
import syllables
import regex


In [4]:
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('cmudict')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

In [5]:
# NOW merge all the stopwords into a single file
stopword_file='StopWords.txt'
with open(stopword_file,'r',encoding='latin-1')as file:
    stopwords=file.read().splitlines()

In [6]:
stopwords

['ERNST',
 'YOUNG',
 'DELOITTE',
 'TOUCHE',
 'KPMG',
 'PRICEWATERHOUSECOOPERS',
 'PRICEWATERHOUSE',
 'COOPERS',
 '',
 'ABOUT',
 'ABOVE',
 'AFTER',
 'AGAIN',
 'ALL',
 'AM',
 'AMONG',
 'AN',
 'AND',
 'ANY',
 'ARE',
 'AS',
 'AT',
 'BE',
 'BECAUSE',
 'BEEN',
 'BEFORE',
 'BEING',
 'BELOW',
 'BETWEEN',
 'BOTH',
 'BUT',
 'BY',
 'CAN',
 'DID',
 'DO',
 'DOES',
 'DOING',
 'DOWN',
 'DURING',
 'EACH',
 'FEW',
 'FOR',
 'FROM',
 'FURTHER',
 'HAD',
 'HAS',
 'HAVE',
 'HAVING',
 'HE',
 'HER',
 'HERE',
 'HERS',
 'HERSELF',
 'HIM',
 'HIMSELF',
 'HIS',
 'HOW',
 'IF',
 'IN',
 'INTO',
 'IS',
 'IT',
 'ITS',
 'ITSELF',
 'JUST',
 'ME',
 'MORE',
 'MOST',
 'MY',
 'MYSELF',
 'NO',
 'NOR',
 'NOT',
 'NOW',
 'OF',
 'OFF',
 'ON',
 'ONCE',
 'ONLY',
 'OR',
 'OTHER',
 'OUR',
 'OURS',
 'OURSELVES',
 'OUT',
 'OVER',
 'OWN',
 'SAME',
 'SHE',
 'SHOULD',
 'SO',
 'SOME',
 'SUCH',
 'THAN',
 'THAT',
 'THE',
 'THEIR',
 'THEIRS',
 'THEM',
 'THEMSELVES',
 'THEN',
 'THERE',
 'THESE',
 'THEY',
 'THIS',
 'THOSE',
 'THROUGH',
 'TO',
 

Loading negative words


In [7]:
negative_words='negative-words.txt'
with open(negative_words,'r',encoding='ISO-8859-1') as file:
    negative_words=file.read().splitlines()

In [8]:
negative_words


['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted',
 'aborts',
 'abrade',
 'abrasive',
 'abrupt',
 'abruptly',
 'abscond',
 'absence',
 'absent-minded',
 'absentee',
 'absurd',
 'absurdity',
 'absurdly',
 'absurdness',
 'abuse',
 'abused',
 'abuses',
 'abusive',
 'abysmal',
 'abysmally',
 'abyss',
 'accidental',
 'accost',
 'accursed',
 'accusation',
 'accusations',
 'accuse',
 'accuses',
 'accusing',
 'accusingly',
 'acerbate',
 'acerbic',
 'acerbically',
 'ache',
 'ached',
 'aches',
 'achey',
 'aching',
 'acrid',
 'acridly',
 'acridness',
 'acrimonious',
 'acrimoniously',
 'acrimony',
 'adamant',
 'adamantly',
 'addict',
 'addicted',
 'addicting',
 'addicts',
 'admonish',
 'admonisher',
 'admonishingly',
 'admonishment',
 'admonition',
 'adulterate',
 'adulterated',
 'adulteration',
 'adulterier',
 'adversarial',
 'adversary',
 'adverse',
 'adversity',
 'afflict',
 'affliction',
 'afflictive',
 'affront',


Loading positive words

In [9]:
positive_words='positive-words.txt'
with open(positive_words,'r',encoding='ISO-8859-1') as file:
    positive_words=file.read().splitlines()

# now using beautifulsoup we have to extract text

In [31]:
#create a function where we extract the data from url

def text_ext(url):
    response=requests.get(url)#we store the request in on reponse
    soup=BeautifulSoup(response.text,'html.parser')#we want the content of the each wabsite (so we choose html.parser)
    text=soup.get_text()
    return text

# now use functon to remove stopwords from the text

In [11]:
def remove_words(text):
    words=nltk.word_tokenize(text) #word tokeniztion is used to create a each token from the sentence
    filter_words=[word for word in words if word.lower() not in stopwords]#it create a section where lower words should not match with the stopwods and create a new words list called filter_word
    return " ".join(filter_words)

#  Function to caluclate sentimental score

In [12]:
def sentiment_score(text):
    senscore=SentimentIntensityAnalyzer()
    sentiment_scores=senscore.polarity_scores(text)
    return sentiment_scores['pos'], sentiment_scores['neg'], sentiment_scores['compound'], sentiment_scores['neu']

# now create a function to find the average sentence length

In [13]:
def average_length(text):
    sentences=nltk.sent_tokenize(text)
    num_sentences=len(sentences)
    total_words=sum(len(nltk.word_tokenize(sentence)) for sentence in sentences)
    return total_words/num_sentences


In [14]:
average_length("Hello! This is an example sentence. And here's another one.")

4.666666666666667

#  Function to Calculate Syllable Count

In [15]:
def tot_syllable(word):
    return syllables.estimate(word)

# Now finding the PERCENTAGE ComplexWords

In [33]:
def complex_words(text):
    words=nltk.word_tokenize(text)
    total_words=len(words)
    complex_word_count=sum(1 for word in words if tot_syllable(word)>2 and tot_syllable(word)!=0)
    return (complex_word_count/total_words)*100

# Now caluclate Fog index

In [19]:
def fog_ind(average_length,complex_words):
    return 0.4*(average_length+complex_words)

# Now we have to focus on personal Prononus

In [38]:
def count_personal(text):
    personal_pronouns=['I', 'me', 'my', 'mine', 'we', 'us', 'our', 'ours']
    pattern=r'\b(?:{})\b'.format('|'.join(personal_pronouns))
    count =len(regex.findall(pattern,text,flags=regex.IGNORECASE))
    return count
    

# Now caluclating Average Word Length

In [21]:
def average_word_le(text):
    words=nltk.word_tokenize(text)
    total_length=sum(len(word) for word in words)
    return total_length/len(words)

# Now input the excel file through pandas

In [25]:
import pandas as pd
df=pd.read_excel("Input.xlsx")

In [26]:
df

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...
...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...
111,51844.6,https://insights.blackcoffer.com/what-are-the-...
112,52306.4,https://insights.blackcoffer.com/marketing-dri...


now we have imported the df and we have to add the columns to make it similar to the output structure

# Add new columns to the df

In [27]:
df['POSITIVE SCORE'] = 0.0
df['NEGATIVE SCORE'] = 0.0
df['POLARITY SCORE'] = 0.0
df['SUBJECTIVITY SCORE'] = 0.0
df['AVG SENTENCE LENGTH'] = 0.0
df['PERCENTAGE OF COMPLEX WORDS'] = 0.0
df['FOG INDEX'] = 0.0
df['AVG NUMBER OF WORDS PER SENTENCE'] = 0.0
df['COMPLEX WORD COUNT'] = 0
df['WORD COUNT'] = 0
df['SYLLABLE PER WORD'] = 0.0
df['PERSONAL PRONOUNS'] = 0
df['AVG WORD LENGTH'] = 0.0

Now we have to  run these url in the iterative mode and caluclate metrics

In [39]:
for i,row in df.iterrows():
    url=row['URL']
    data=text_ext(url)
    data=remove_words(data)
    positive_score, negative_score, polarity_score, subjective_score=sentiment_score(data)
    avg_sentence_length=average_length(data)
    word_tokens=nltk.word_tokenize(data)
    percentage_complex_words=complex_words(data)
    fog_index=fog_ind(avg_sentence_length,percentage_complex_words)
    complex_word_count=sum(1 for word in nltk.word_tokenize(data) if tot_syllable(word) > 2)
    word_count = len(nltk.word_tokenize(data))
    syllable_per_word=sum(tot_syllable(word) for word in nltk.word_tokenize(data))/word_count
    personal_pronouns_count=count_personal(data)
    avg_word_length=average_word_le(data)
    #-----------------------------------------------------------------#
    #adding the data to the give columns#
    df.at[i, 'POSITIVE SCORE'] = positive_score
    df.at[i, 'NEGATIVE SCORE'] = negative_score
    df.at[i, 'POLARITY SCORE'] = polarity_score
    df.at[i, 'SUBJECTIVITY SCORE'] = subjective_score
    df.at[i, 'AVG SENTENCE LENGTH'] = avg_sentence_length
    df.at[i, 'PERCENTAGE OF COMPLEX WORDS'] = percentage_complex_words
    df.at[i, 'FOG INDEX'] = fog_index
    df.at[i, 'AVG NUMBER OF WORDS PER SENTENCE'] = word_count / len(nltk.sent_tokenize(data))
    df.at[i, 'COMPLEX WORD COUNT'] = complex_word_count
    df.at[i, 'WORD COUNT'] = word_count
    df.at[i, 'SYLLABLE PER WORD'] = syllable_per_word
    df.at[i, 'PERSONAL PRONOUNS'] = personal_pronouns_count
    df.at[i, 'AVG WORD LENGTH'] = avg_word_length

#  Save the file into Excel file

In [40]:
df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,0.226,0.025,0.9999,0.750,19.187500,40.933768,24.048507,19.187500,754,1842,2.409881,0,6.460912
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,0.224,0.011,0.9996,0.765,33.352941,39.329806,29.073099,33.352941,446,1134,2.351852,0,6.405644
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,0.169,0.040,0.9993,0.791,19.675325,35.313531,21.995542,19.675325,535,1515,2.247525,0,6.139934
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,0.225,0.042,0.9997,0.733,22.013889,37.034700,23.619436,22.013889,587,1585,2.265615,0,6.229653
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,0.227,0.041,0.9998,0.732,22.457143,37.086514,23.817463,22.457143,583,1572,2.269720,0,6.246183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,0.119,0.039,0.9978,0.843,31.725000,33.412136,26.054854,31.725000,424,1269,2.227738,0,6.022065
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,0.122,0.092,0.9974,0.786,31.500000,30.926779,24.970712,31.500000,604,1953,2.142857,0,5.827445
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,0.199,0.033,0.9998,0.768,23.337349,35.209086,23.418574,23.337349,682,1937,2.207537,0,6.062984
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,0.150,0.062,0.9988,0.788,21.853333,33.312996,22.066532,21.853333,546,1639,2.203783,0,5.969494


In [41]:
df.to_excel('output.xlsx', index=False)